# 量化矩阵相乘

In [34]:
import numpy as np
from concrete.ml.quantization.quantizers import(
    QuantizedArray,
    MinMaxQuantizationStats,
    QuantizationOptions,
)

## 创建两个矩阵

In [35]:
# 创建两个浮点数矩阵
matrix1 = np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float64)
matrix2 = np.array([[5.0, 6.0], [7.0, 8.0]], dtype=np.float64)

# 使用numpy.dot函数计算矩阵相乘的结果
result_dot = np.dot(matrix1, matrix2)

# 或者使用@运算符计算矩阵相乘的结果
result_at = matrix1 @ matrix2

print("矩阵1:")
print(matrix1)

print("\n矩阵2:")
print(matrix2)

矩阵1:
[[1. 2.]
 [3. 4.]]

矩阵2:
[[5. 6.]
 [7. 8.]]


## 将矩阵1和矩阵2量化

In [36]:
options=QuantizationOptions(8,is_symmetric=True,is_signed=True)
q_matrix1 = QuantizedArray(8,matrix1,options=options)
q_matrix2 = QuantizedArray(8,matrix2,options=options)
print("q_matrix1:",q_matrix1.qvalues)
print("q_matrix2:",q_matrix2.qvalues)

q_matrix1: [[ 32  64]
 [ 95 127]]
q_matrix2: [[ 79  95]
 [111 127]]


矩阵量化参数

In [37]:
print("q_matrix1:",q_matrix1.quantizer.dump_dict())
print("*"*100)
print("q_matrix2:",q_matrix2.quantizer.dump_dict())

q_matrix1: {'n_bits': 8, 'is_signed': True, 'is_symmetric': True, 'is_qat': False, 'is_narrow': False, 'is_precomputed_qat': False, 'rmax': 4.0, 'rmin': 1.0, 'uvalues': array([1., 2., 3., 4.]), 'scale': 0.031496062992125984, 'zero_point': 0, 'offset': 128, 'no_clipping': False}
****************************************************************************************************
q_matrix2: {'n_bits': 8, 'is_signed': True, 'is_symmetric': True, 'is_qat': False, 'is_narrow': False, 'is_precomputed_qat': False, 'rmax': 8.0, 'rmin': 5.0, 'uvalues': array([5., 6., 7., 8.]), 'scale': 0.06299212598425197, 'zero_point': 0, 'offset': 128, 'no_clipping': False}


## 量化矩阵相乘

In [38]:
q_result = q_matrix1.qvalues @ q_matrix2.qvalues
print("q_result:",q_result)

q_result: [[ 9632 11168]
 [21602 25154]]


## 反量化矩阵相乘的结果

In [39]:
result = q_result*q_matrix1.quantizer.scale * q_matrix2.quantizer.scale
print("result:",result)

result: [[19.10992622 22.15735631]
 [42.85845372 49.90563581]]


量化矩阵相乘的结果

In [40]:
print("real_result:",q_matrix1.values @ q_matrix2.values)

real_result: [[19. 22.]
 [43. 50.]]


## 使用QuantizedMatMul计算矩阵相乘

In [41]:
from concrete.ml.quantization.quantized_ops import (
    QuantizedMatMul,
)

In [42]:
q_mm=QuantizedMatMul(
    8,
    "Test_"+"QuantizedMatmul",
    int_input_names={"0"},
    constant_inputs={"b": q_matrix2},
)
expected_mm_outputs = q_mm.calibrate(matrix1)
print("expected_gemm_outputs:",expected_mm_outputs)
actual_mm_outputs = q_mm(q_matrix1)
print("actual_gemm_outputs:",actual_mm_outputs.dequant())

expected_gemm_outputs: [[19. 22.]
 [43. 50.]]
actual_gemm_outputs: [[18.96470588 22.00392157]
 [42.91372549 49.96470588]]


## 使用QuantizedGemm计算矩阵相乘

In [43]:
from concrete.ml.quantization.quantized_ops import (
    QuantizedGemm,
)

In [44]:
q_gemm = QuantizedGemm(
        8,
        "Test_" + "QuantizedGemm",
        int_input_names={"0"},
        constant_inputs={"b": q_matrix2},
    )
expected_gemm_outputs = q_gemm.calibrate(matrix1)
print("expected_gemm_outputs:",expected_gemm_outputs)
actual_gemm_outputs = q_gemm(q_matrix1)
print("actual_gemm_outputs:",actual_gemm_outputs.dequant())

expected_gemm_outputs: [[19. 22.]
 [43. 50.]]
actual_gemm_outputs: [[18.96470588 22.00392157]
 [42.91372549 49.96470588]]


## 将矩阵相乘编译成电路

In [46]:
from concrete import fhe
import numpy as np

In [49]:
weight=np.random.rand(576,10)
bias=np.random.rand(10)
print("weight:",weight)
print("bias:",bias)


weight: [[0.95465017 0.58393463 0.35734682 ... 0.22489583 0.55098736 0.3623464 ]
 [0.09161431 0.89189404 0.83573627 ... 0.88843133 0.26682027 0.2463664 ]
 [0.97754837 0.10080221 0.95624031 ... 0.45348708 0.82623286 0.94490637]
 ...
 [0.842556   0.22946084 0.27145455 ... 0.20887805 0.39179828 0.47013186]
 [0.39623873 0.51532867 0.94003651 ... 0.09512315 0.75648157 0.0526663 ]
 [0.73702707 0.34416041 0.74220157 ... 0.87376436 0.88828931 0.03447776]]
bias: [0.6076683  0.18899815 0.8529939  0.54036791 0.09063075 0.18266463
 0.17376126 0.96365516 0.70750656 0.1754963 ]


In [50]:
options=QuantizationOptions(8,is_symmetric=True,is_signed=True)
q_weight=QuantizedArray(8,weight,options=options)
q_bias=QuantizedArray(32,bias,options=options)
print("q_weight:",q_weight.qvalues)
print("q_bias:",q_bias.qvalues)

q_weight: [[121  74  45 ...  29  70  46]
 [ 12 113 106 ... 113  34  31]
 [124  13 121 ...  58 105 120]
 ...
 [107  29  34 ...  27  50  60]
 [ 50  65 119 ...  12  96   7]
 [ 94  44  94 ... 111 113   4]]
q_bias: [1354175006  421178079 1900877544 1204197616  201968565  407063984
  387223020 2147483647 1576662297  391089526]


In [54]:
configuration = fhe.Configuration(
    enable_unsafe_features=True,
    show_mlir=False,
    show_graph=True,
)
@fhe.compiler({"q_x": "encrypted"})
def f_lr(q_x):
    res = q_x @ q_weight.qvalues
    return res

In [63]:
inputset = [np.random.randint(-128, 128, size=(576,)) for i in range(10000)]

In [64]:
circuit = f_lr.compile(inputset,configuration=configuration)


Computation Graph
--------------------------------------------------------------------------------
%0 = q_x                              # EncryptedTensor<int9, shape=(576,)>        ∈ [-128, 255]
%1 = [[121  74  ...  113   4]]        # ClearTensor<uint7, shape=(576, 10)>        ∈ [0, 127]
%2 = matmul(%0, %1)                   # EncryptedTensor<int24, shape=(10,)>        ∈ [-537182, 5391143]
return %2
--------------------------------------------------------------------------------



In [65]:
input=np.random.randn(576,)
print("input:",input.shape)

input: (576,)


In [66]:
q_input=QuantizedArray(8,input,options=options)
print("q_input:",q_input.qvalues)

q_input: [ -60    4  -39   66  -43  -17   -4  -68   -5  -12  -71  -13   35  -52
    0  -54   -3  -72   10   -7   52  -47  -54    3   37  -45   42   51
   29  102  -21  -15   45   69  -65   46   -3   79   23   40   66   27
   52   77   32   13   14  -40  -45   20  -33  -43   -6  -23    1   42
    9    1  -55   49   -7   19  -58   30  -78   37   43  -16  -30   22
   28   -5  -13   22   37  -60    7  -29   24   31   25   43  -58   93
   20   13  -87  -11   56  -44   31  -53  -21   -2    4   37  -15   19
  -10   36  -52  -48   11   -1  -73  -17   52  -35    5  -44  -16   37
   58  -53   48  -16    8  -54  -24  -13  -64  -31   -7   -2  -80    8
   12   -7  -37   -2  -16    6   47   25    2  -38   -5  -71  -35   44
  -70   15  -52   41   67   -1   10   74   -5   73  -75   -3  -33  -67
    5   27   45   20    4  -10   44    2  -18   40  -54   30  -21   -1
   57   77    3  -33   18  -44   55   16    5   34   10   50   -1   81
   21   -7  -52   33    4  -75  -84   24  -83   51  125  -32   20  -

In [71]:
q_result=circuit.encrypt_run_decrypt(q_input.qvalues)
deq_result=q_result*q_weight.quantizer.scale*q_input.quantizer.scale
print("deq_result:",deq_result)
print("real_result:",input@weight)

deq_result: [ 6.75702657 -5.09449915 -0.69947311  5.48305074  5.34564349 -1.26695502
 -9.94347059  2.11847878  5.94201101  7.44707173]
real_result: [ 6.72275252 -5.09005379 -0.73747859  5.33244733  5.32704307 -1.14732623
 -9.94018604  2.0951329   5.84514328  7.3201499 ]
